In [ ]:
import pandas as pd  # 导入 pandas 库用于数据处理
import numpy as np  # 导入 numpy 库用于数值计算
import torch  # 导入 PyTorch 库用于深度学习模型
import torchvision  # 导入 torchvision 库用于计算机视觉任务
import os  # 导入 os 库用于文件操作
import torchvision.transforms as transforms  # 导入 torchvision 的图像预处理模块
from torchvision.datasets import ImageFolder  # 导入 ImageFolder 用于加载图像数据集
from torch.utils.data import DataLoader, Subset, Dataset  # 导入 PyTorch 的数据加载器和子集工具
from PIL import Image  # 导入 PIL 用于处理图像
# !cp -r "/kaggle/input/rsna-models/facebookresearch_dinov2_main (1)/root/.cache/torch/hub/facebookresearch_dinov2_main" /kaggle/working/dinov2  # 将模型缓存拷贝到当前目录

mean = [0.485, 0.456, 0.406]  # DINO 使用的图像均值
std = [0.229, 0.224, 0.225]  # DINO 使用的图像标准差

from transformers import AutoImageProcessor, AutoModel  # 导入 Hugging Face 的 DINO 模型与处理器

# 加载 DINO 预训练模型和图像处理器
processor = AutoImageProcessor.from_pretrained('/kaggle/input/dinov2/pytorch/giant/1')
model = AutoModel.from_pretrained('/kaggle/input/dinov2/pytorch/giant/1')
model = model.cuda()  # 将模型转移到 GPU 上进行计算

# 初始化特征提取和目标存储的容器
embeds = []  # 存储提取的图像特征
targets = [[] for i in range(5)]  # 存储每个目标的实际值（有 5 个目标）
counter = 0  # 计数器，用于记录处理的图像批次数

# 读取训练数据集
train_df = pd.read_csv("/kaggle/input/csiro-biomass/train.csv")
root = "/kaggle/input/csiro-biomass/"

# 遍历训练数据集，提取图像特征
for i in range(len(train_df)):
    entry = train_df.iloc[i]  # 获取当前行数据
    file_path = root + entry['image_path']  # 获取图像文件路径
    y = torch.tensor([[entry['target']]])  # 获取目标值
    targets[i % 5].append(y)  # 将目标值按目标列分配
    if i % 5 == 0:  # 每处理一张图像时执行一次
        img = Image.open(file_path)  # 打开图像文件
        x = torch.tensor(processor(img).pixel_values)  # 将图像转换为模型输入格式
        with torch.no_grad():  # 禁用梯度计算
            x = x.cuda()  # 将图像数据转移到 GPU
            embeds.append(model(x).pooler_output.cpu())  # 提取特征并移动回 CPU
            counter += 1  # 增加计数器
            if counter % 100 == 0:  # 每处理 100 张图像时打印进度
                print(f"{counter} batches processed.")  # 打印进度信息
                
# 数据分割和交叉验证的初始化
import random
import numpy as np
from sklearn.linear_model import Lasso  # 导入 Lasso 回归模型
from sklearn.metrics import r2_score  # 导入 R² 评分计算

# 创建索引并随机打乱
lst = list(range(len(embeds)))  # 将索引列表化
random.seed(42)  # 设置随机种子
random.shuffle(lst)  # 打乱数据

# 创建多个 80/20 的随机划分
n_splits = 5  # 设置交叉验证折数为 5
splits = []  # 存储每折的训练和验证索引

for i in range(n_splits):  # 创建 5 折交叉验证
    temp_lst = lst.copy()  # 复制打乱的索引列表
    random.seed(42 + i)  # 为每折使用不同的种子
    random.shuffle(temp_lst)  # 打乱数据
    split_point = int(len(temp_lst) * 0.8)  # 划分训练集和验证集的比例
    train_idxs = temp_lst[:split_point]  # 获取训练集索引
    val_idxs = temp_lst[split_point:]  # 获取验证集索引
    splits.append((train_idxs, val_idxs))  # 将训练集和验证集的索引添加到 splits

# 转换 embeds 为 numpy 数组以提高效率
embeds_np = np.array(torch.cat(embeds))  # 将特征合并为一个 NumPy 数组
regressors = [[None for i in range(5)] for j in range(5)]  # 初始化回归器列表，用于存储每个目标的模型

# 遍历每个目标进行回归
for i in range(5):
    print(f"\n=== Target {i+1} ===")  # 输出当前目标编号
    targets_np = np.array(torch.cat(targets[i]))  # 获取当前目标的目标值
    
    split_scores = []  # 存储每折的 R² 评分

    for split_idx, (train_idxs, val_idxs) in enumerate(splits):  # 遍历每一折
        print(f"Fold {split_idx+1}:")  # 输出当前折次

        # 获取训练集和验证集的数据
        X_train, y_train = embeds_np[train_idxs], targets_np[train_idxs]
        X_val, y_val = embeds_np[val_idxs], targets_np[val_idxs]

        # 使用 Lasso 回归模型进行训练
        reg = Lasso()  # 初始化 Lasso 回归模型
        reg.fit(X_train, y_train)  # 训练模型

        # 预测并计算训练集和验证集的 R² 分数
        train_preds = reg.predict(X_train)  # 对训练集进行预测
        train_preds[train_preds < 0.0] = 0.0  # 将负值裁剪为 0
        train_r2 = r2_score(y_train, train_preds)  # 计算训练集 R²

        val_preds = reg.predict(X_val)  # 对验证集进行预测
        val_preds[val_preds < 0.0] = 0.0  # 将负值裁剪为 0
        val_r2 = r2_score(y_val, val_preds)  # 计算验证集 R²

        print(f"  Train R²: {train_r2:.4f}")  # 输出训练集 R²
        print(f"  Val R²: {val_r2:.4f}")  # 输出验证集 R²

        split_scores.append((train_r2, val_r2))  # 保存当前折的评分
        regressors[i][split_idx] = reg  # 保存当前折的回归器
    
    # 输出当前目标的平均 R² 分数
    avg_train_r2 = np.mean([score[0] for score in split_scores])  # 计算训练集平均 R²
    avg_val_r2 = np.mean([score[1] for score in split_scores])  # 计算验证集平均 R²

    print(f"\nTarget {i+1} Average:")  # 输出当前目标的平均值
    print(f"  Avg Train R²: {avg_train_r2:.4f}")  # 输出训练集平均 R²
    print(f"  Avg Val R²: {avg_val_r2:.4f}")  # 输出验证集平均 R²

# 目标映射字典
mapping = {"Dry_Clover_g": 0, "Dry_Dead_g": 1, "Dry_Green_g": 2, "Dry_Total_g": 3, "GDM_g": 4}

# 测试集特征提取与预测
test_embeds = {}  # 存储测试集的图像特征
counter = 0  # 计数器

# 读取测试数据
test_df = pd.read_csv("/kaggle/input/csiro-biomass/test.csv")
sample_ids = []  # 存储样本 ID

# 提取测试集图像特征
for i in range(len(test_df)):
    entry = test_df.iloc[i]  # 获取当前行数据
    file_path = root + entry['image_path']  # 获取图像文件路径
    sample_id = entry['sample_id']  # 获取样本 ID
    if sample_id not in sample_ids:  # 确保每个样本只处理一次
        img = Image.open(file_path)  # 打开图像文件
        x = torch.tensor(processor(img).pixel_values)  # 将图像转换为模型输入格式
        with torch.no_grad():  # 禁用梯度计算
            x = x.cuda()  # 将图像数据转移到 GPU
            test_embeds[sample_id.split("_")[0]] = model(x).pooler_output.cpu()  # 提取特征并保存
            counter += 1  # 增加计数器
        sample_ids.append(sample_id)  # 添加样本 ID
    if counter % 100 == 0:  # 每处理 100 张图像时打印进度
        print(f"{counter} batches processed.")  # 打印进度信息

# 预测测试集
predictions = []  # 存储预测结果
sample_ids = []  # 存储样本 ID

# 遍历测试数据进行预测
for i in range(len(test_df)):
    try:
        entry = test_df.iloc[i]  # 获取当前行数据
        X = np.array(test_embeds[entry['sample_id'].split("__")[0]])  # 获取测试样本特征
        sample_ids.append(entry['sample_id'])  # 存储样本 ID
        models = regressors[mapping[entry['sample_id'].split("__")[1]]]  # 获取当前目标的回归模型
        prediction = 0.0  # 初始化预测值
        for item in models:  # 对每个模型进行预测并加权平均
            single_pred = item.predict(X)
            if single_pred < 0.0:  # 若预测值为负则裁剪为 0
                single_pred = 0.0
            prediction += single_pred
        prediction = prediction / 5  # 平均 5 折预测值
        predictions.append(float(prediction))  # 存储预测结果
    except Exception as e:  # 异常处理
        predictions.append(0.0)  # 出现异常时预测为 0

# 生成提交文件
# %cd /kaggle/working  # 切换到当前工作目录

submission = pd.DataFrame({
    'sample_id': sample_ids,  # 样本 ID
    'target': predictions  # 预测结果
})

# submission.to_csv('submission.csv', index=False)  # 保存为 CSV 文件
# submission  # 输出提交数据


In [ ]:
# 用来实现用公式计算 dead ，然后再把处理好的提交
# 创建一个字典来存储样本 ID 与目标值的映射
target_dict = {}
for i in range(len(submission)):
    sample_id = submission.loc[i, 'sample_id']
    target_value = submission.loc[i, 'target']
    target_dict[sample_id] = target_value  # 将样本 ID 与预测目标值存入字典

# 计算 Dry_Dead_g：根据公式 Dry_Dead_g = Dry_Total_g - GDM_g
for i in range(len(submission)):
    sample_id = submission.loc[i, 'sample_id']
    
    # 使用正则表达式提取样本 ID 和目标
    sample_prefix = sample_id.split("__")[0]
    target = sample_id.split("__")[1]

    # 如果是 Dry_Dead_g，计算值
    if target == "Dry_Dead_g":
        dry_total_g = target_dict.get(f"{sample_prefix}__Dry_Total_g", 0)
        gdm_g = target_dict.get(f"{sample_prefix}__GDM_g", 0)
        # 计算 Dry_Dead_g = Dry_Total_g - GDM_g，并修正为非负
        dead_value = max(dry_total_g - gdm_g, 0)
        submission.loc[i, 'target'] = dead_value  # 更新 Dry_Dead_g 的预测值

# 确保每个目标为非负
submission['target'] = submission['target'].apply(lambda x: max(x, 0))

# print(submission.head())  # 打印前几行确认结果


In [ ]:
# print(submission['Dry_Dead_g'] )


In [ ]:
# # 修改：将 Dry_Dead_g 计算为 Dry_Total_g - GDM_g
# # 通过公式计算 Dry_Dead_g
# submission['Dry_Dead_g'] = submission['Dry_Total_g'] - submission['GDM_g']

# # 如果 Dry_Dead_g 小于 0，则将其置为 0
# submission['Dry_Dead_g'] = submission['Dry_Dead_g'].apply(lambda x: max(x, 0))

In [ ]:
%cd /kaggle/working  # 切换到当前工作目录
submission.to_csv('submission.csv', index=False)  # 保存为 CSV 文件
submission  # 输出提交数据